In [104]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

# Загрузка данных

In [105]:
%%capture
!wget https://www.dropbox.com/s/64ol9q9ssggz6f1/data_ford_price.xlsx

In [106]:
data = pd.read_excel('data/data_ford_price.xlsx') 

#  Отбор признаков: мотивация

## Предобработка данных

In [107]:
data = data[['price','year', 'cylinders', 'odometer', 'lat', 'long', 'weather']]
data.dropna(inplace = True)

y = data['price']
x = data.drop(columns='price')

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

## Обучение модели

In [108]:
model = LinearRegression()
model.fit(X_train, y_train)
y_predicted = model.predict(X_test)
 
mae = mean_absolute_error(y_test, y_predicted)
print('MAE: %.3f' % mae)

MAE: 4682.957


## Удаление избыточного признака

In [109]:
x.drop('lat', axis = 1, inplace = True)

In [110]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

In [111]:
model = LinearRegression()
model.fit(X_train, y_train)
y_predicted = model.predict(X_test)
 
mae = mean_absolute_error(y_test, y_predicted)
print('MAE: %.3f' % mae)

MAE: 4672.930


#  Отбор признаков: классификация методов

## Метод рекурсивного исключения признаков

Метод рекурсивного исключения признаков (RFE) предполагает выбор признаков путём рекурсивного рассмотрения всё меньших и меньших наборов фичей.

Сначала RFE обучается на изначальной выборке, и происходит оценка важности каждого признака. Затем наименее важные фичи удаляются. Эта процедура рекурсивно повторяется на сокращённом наборе до тех пор, пока в конечном итоге не будет достигнуто желаемое количество признаков в выборке.

In [112]:
from sklearn.feature_selection import RFE

In [113]:
y = data['price']
x = data.drop(columns='price')

In [114]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

Выделим три наиболее значимых признака:

In [115]:
estimator = LinearRegression()
selector = RFE(estimator, n_features_to_select=3, step=1)
selector = selector.fit(X_train, y_train)
# выделены три столбца-признака для обучения, выбранные RFE
selector.get_feature_names_out()

array(['year', 'cylinders', 'lat'], dtype=object)

Также узнаем, как RFE проранжировал все доступные признаки:

In [116]:
X_train.columns

Index(['year', 'cylinders', 'odometer', 'lat', 'long', 'weather'], dtype='object')

In [117]:
selector.ranking_

array([1, 1, 4, 1, 3, 2])

##  МЕТОДЫ ВЫБОРА ПРИЗНАКОВ НА ОСНОВЕ ФИЛЬТРОВ

В качестве фильтров для выбора признаков используются уже знакомые нам статистики, такие как коэффициент корреляции Пирсона, ANOVA и т. д. При этом выбор статистических показателей сильно зависит от типов переменных в данных.

Чем больше известно о типе данных, тем проще выбрать подходящую статистическую меру для метода отбора признаков на основе фильтра. Ниже приведена схема-помощник в выборе метода селекции признаков.

In [118]:
from sklearn.feature_selection import SelectKBest, f_regression

In [119]:
selector = SelectKBest(f_regression, k=3)
selector.fit(X_train, y_train)
 
selector.get_feature_names_out()

array(['year', 'cylinders', 'odometer'], dtype=object)

Библиотека sklearn обеспечивает реализацию большинства полезных статистических показателей, например:

- коэффициента корреляции Пирсона: f_regression();
- дисперсионного анализа ANOVA: f_classif();
- хи-квадрата: chi2();
- взаимной информации: mutual_info_classif() и mutual_info_regression().

Кроме того, библиотека SciPy обеспечивает реализацию многих других статистических данных, таких как тау Кендалла (kendalltau) и ранговая корреляция Спирмена (spearmanr).

sklearn также предоставляет множество различных методов фильтрации после расчёта статистики для каждой входной переменной с целевой.

Два наиболее популярных метода:

- выбор k лучших переменных: SelectKBest;
- выбор переменных верхнего процентиля: SelectPercentile.

## Задание 9.5. Модуль ML-6 (HW-03)
Обучите модель линейной регрессии на найденных двумя способами трёх важных признаках и сравните полученные результаты.

- Верно выделены три столбца-признака для обучения, выбранные RFE.
-	Верно выделены три столбца-признака для обучения, выбранные SelectKBest.
-	Обучена регрессия на первых трёх столбцах, оценено качество модели на тесте.
-	Обучена регрессия на вторых трёх столбцах, оценено качество модели на тесте.
-	Произведено сравнение выбранных метрик в форме комментария. Дан ответ на вопрос «Какой метод отбора признаков показал наилучший результат на тестовой выборке?» (в текстовой ячейке).

Выделяем три столбца-признака для обучения, выбранные RFE

In [120]:
estimator = LinearRegression()
rfe_selector = RFE(estimator, n_features_to_select=3, step=1)
rfe_selector = rfe_selector.fit(X_train, y_train)
# выделены три столбца-признака для обучения, выбранные RFE
rfe_cols = rfe_selector.get_feature_names_out()
display(rfe_cols)

array(['year', 'cylinders', 'lat'], dtype=object)

Выделяем три столбца-признака для обучения, выбранные SelectKBest

In [121]:
skb_selector = SelectKBest(f_regression, k=3)
skb_selector.fit(X_train, y_train)
# выделены три столбца-признака для обучения, выбранные SelectKBest
skb_cols = selector.get_feature_names_out()
display(skb_cols)

array(['year', 'cylinders', 'odometer'], dtype=object)

Обучаем регрессию на первых трёх столбцах, оценено качество модели на тесте

In [122]:
# выделение целевой переменной и признаков
y = data['price']
x = data[rfe_cols]
# разделение данных на тренировочные и тестовые наборы
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)
# создание и обучение модели линейной регрессии
rfe_model = LinearRegression()
rfe_model = rfe_model.fit(X_train, y_train)
# предсказание значений на тестовом наборе
y_test_pred = rfe_model.predict(X_test)
# выводим оценку качества модели
print(' RFE Test MAE: {:.3f}'.format(round(mean_absolute_error(y_test, y_test_pred))))

 RFE Test MAE: 5097.000


Обучаем регрессию на вторых трёх столбцах, оценено качество модели на тесте

In [123]:
# выделение целевой переменной и признаков
y = data['price']
x = data[skb_cols]
# разделение данных на тренировочные и тестовые наборы
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)
# создание и обучение модели линейной регрессии
lin_reg = LinearRegression()
lin_reg = lin_reg.fit(X_train, y_train)
# предсказание значений на тестовом наборе
y_test_pred = lin_reg.predict(X_test)
# выводим оценку качества модели
print('SCB Test MAE: {:.3f}'.format(round(mean_absolute_error(y_test, y_test_pred))))

SCB Test MAE: 4709.000


Производим сравнение выбранных метрик в форме комментария:

Для оценки результатов методов отбора признаков RFE (Recursive Feature Elimination) и SCB (Some Custom Baseline) были использованы метрики MAE (Mean Absolute Error) на тестовой выборке. MAE - это среднее абсолютное отклонение предсказанных значений от истинных значений, и оно используется для оценки точности модели.

Сравнивая метрики MAE для каждого метода:

- RFE Test MAE: 5097.000

- SCB Test MAE: 4709.000

Метод SCB показал меньшую величину MAE по сравнению с методом RFE, что указывает на то, что метод SCB имеет более точные предсказания (меньшее отклонение от истинных значений) на тестовой выборке.

Ответ: Метод отбора признаков SCB показал наилучший результат на тестовой выборке, так как его метрика MAE (4709.000) меньше, чем у метода RFE (5097.000).